In [1]:
import ISRAnalysis as an

Welcome to JupyROOT 6.14/04


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys

# Unfolding for detector (up to dressed lepton level) 

In [25]:
binDef = "_FineCoarse"
channel = "electron"
year = "2016"
doSys = True

# Redirect STDOUT and STDERR
sys.stderr = open("./log/stderr_"+channel+"_"+year+".txt", "w")
sys.stdout = open("./log/stdout_"+channel+"_"+year+".txt", "w")

detUnfold = an.ISRAnalysis("DetUNFOLD", year, channel, 0, False, "matrix", 
                                     "Detector_Dressed_DRp1_Fiducial", "Detector_Dressed_DRp1", binDef)


In [26]:
# Nominal
detUnfold.setInputHist(False, False, None, "Detector")
detUnfold.setUnfoldBkgs(False, "Detector")
detUnfold.subFake() # Subract DY fake (out of fiducial volume at gen level)
#detUnfold_muon_2016.subFake(False, "detector_level_DY_Fake") # Subract DY fake (out of fiducial volume at gen level)

In [27]:
if doSys == True:
    if channel == "muon" :
        systematics = {
                       "ID":["IdSFUp", "IdSFDown"], 
                       "ISO":["IsoSFUp", "IsoSFDown"], 
                       "TRG":["trgSFUp", "trgSFDown"], 
                       #"L1Prefie":["L1PrefireUp","L1PrefireDown"], 
                       "PU":["PUUp", "PUDown"],
                       #"#ZptCorr": ["ZpTCorrected", "Nominal"], # use different matrix file
                       #"LepScale": ["LepScaleUp", "LepScaleDown"],
                       "Unfolding": ["unfoldIterEM", "unfoldNominal"],
                       "Scale":["ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"], 
                       "AlphaS":["AlphaSUp", "AlphaSDown"],
                       #"PDF":["PDFerror{:0>3}".format(str(i)) for i in range(1,101)],
                       "Background": ["NormUp", "NormDown"],
                       }
    
    if channel == "electron" :
        systematics = {
                       "ID":["IdSFUp", "IdSFDown"],
                       "Reco":["recoSFUp","recoSFDown"], 
                       "TRG":["trgSFUp", "trgSFDown"], 
                       #"L1Prefie":["L1PrefireUp","L1PrefireDown"], 
                       "PU":["PUUp", "PUDown"],
                       #"ZptCorr": ["ZpTCorrected", "Nominal"],
                       #"LepScale": ["LepScaleUp", "LepScaleDown"],
                       #"LepRes": ["LepResUp", "LepResDown"],
                       "Unfolding": ["unfoldIterEM", "unfoldNominal"],
                       "Scale":["ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"], 
                       "AlphaS":["AlphaSUp", "AlphaSDown"],
                       #"PDF":["PDFerror{:0>3}".format(str(i)) for i in range(1,101)],
                       "Background": ["NormUp", "NormDown"],
                        }
    
    for sysName, postfixs in systematics.items():
        for postfix in postfixs:
            detUnfold.setSystematics(sysName, postfix)
            #print(sysName + " " + postfix)
            detUnfold.setInputHist(False, False, None, "Detector", True, sysName, postfix)
            detUnfold.setUnfoldBkgs(True, "Detector", sysName, postfix) #Combine Bkgs and Fake?
            histpostfix = postfix
            if sysName == "Unfolding" or "LepScale" in sysName or "LepRes" in sysName:
                histpostfix = ""
            detUnfold.subFake(True, "Detector_DY_Fake", sysName, postfix, histpostfix)
            #detUnfold_.subFake(True, "detector_level_DY_Fake", sysName, postfix, histpostfix)

In [28]:
detUnfold.drawResponseM("Pt")
detUnfold.drawResponseM("Mass")

detUnfold.checkMatrixCond("Pt")
detUnfold.checkMatrixCond("Mass")

<ROOT.TVectorT<double> object ("TVectorT<double>") at 0xa869dc8>

In [7]:
detUnfold.doUnfold() # nominal unfold
if doSys:
    detUnfold.doUnfold(doSys)

# Set mean values


In [8]:
print("set mean values")
detUnfold.setMeanValues(True)
if doSys:
    detUnfold.setSysMeanValues()
    detUnfold.setSysError()
    detUnfold.setTotSysError()

In [9]:
detUnfold.doAcceptance(doSys, False, "DetEfficienyCorr")

In [10]:
detUnfold.setAcceptMeanValues()
if doSys:
    detUnfold.setAcceptSysMeanValues()
    detUnfold.setAcceptSysError()
    detUnfold.setAcceptTotSysError()

# Save to CSV files

In [11]:
#pd_det_mass, pd_det_pt = detUnfold.makeMeanDataFrames("Detector", systematics)
#pd_unf_mass, pd_unf_pt = detUnfold.makeMeanDataFrames("Unfolded", systematics)
#pd_acc_mass, pd_acc_pt = detUnfold.makeMeanDataFrames("Acceptance", systematics)

In [12]:
#pd_det_mass.to_csv(year+"_"+channel+"_mass_det.csv")
#pd_det_pt.to_csv(year+"_"+channel+"_pt_det.csv")
#
#pd_unf_mass.to_csv(year+"_"+channel+"_mass_det_unf.csv")
#pd_unf_pt.to_csv(year+"_"+channel+"_pt_det_unf.csv")
#
#pd_acc_mass.to_csv(year+"_"+channel+"_mass_det_acc.csv")
#pd_acc_pt.to_csv(year+"_"+channel+"_pt_det_acc.csv")

# Unfolding for QED FSR

In [13]:
binDef = "_CoarseCoarse"
phase_space = "Fiducial"

fsrUnfold = an.ISRAnalysis("FSRUNFOLD", year, channel, 0, True, "fsr_matrix", 
                                     "Dressed_DRp1_Dressed_DR4PI_"+phase_space, "Dressed_DRp1_Dressed_DR4PI", binDef)

In [14]:
fsrUnfold.setInputHist(False, True, detUnfold.getISRUnfold()) # Set nominal input using previous unfolding result
fsrUnfold.subFake(False, "Dressed_DRp1_DY_Fake", "Nominal", "", "", True)

In [29]:
fsrUnfold.drawResponseM("Pt")
fsrUnfold.drawResponseM("Mass")

fsrUnfold.checkMatrixCond("Pt")
fsrUnfold.checkMatrixCond("Mass")

<ROOT.TVectorT<double> object ("TVectorT<double>") at 0x88abf98>

In [16]:
if doSys :
    fsr_systematics = {
                       "Scale":["ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown"], "AlphaS":["AlphaSUp", "AlphaSDown"],
                       #"PDF":["PDFerror{:0>3}".format(str(i)) for i in range(1,101)],
                       "Unfolding": ["unfoldIterEM", "unfoldNominal"],
                       "FSR": ["fsrPHOTOS","fsrPYTHIA"],
    }
    
    for sysName, postfixs in fsr_systematics.items():
        for postfix in postfixs:
            fsrUnfold.setSystematics(sysName, postfix, True) # Create TUnfoldDensity objects for systematics
            
            if "FSR" in sysName:
                fsrUnfold.setInputHist(False, True, detUnfold.getISRUnfold(), "", True, sysName, postfix)
                fsrUnfold.subFake(True, "Dressed_DRp1_DY_Fake", sysName, postfix, postfix, True)

In [17]:
fsrUnfold.setFromPreviousUnfold(detUnfold.getISRUnfold()) # Set using previous unfolding result for systematics

In [18]:
if doSys:
    for sysName, postfixs in systematics.items():
        for postfix in postfixs:
            fsrUnfold.subFake(True, "Dressed_DRp1_DY_Fake", sysName, postfix, "", True)

In [19]:
fsrUnfold.doUnfold() # do nominal unfold
if doSys : 
    fsrUnfold.doUnfold(doSys) # do unfold for systematic

In [20]:
fsrUnfold.setMeanValues(True)
if doSys:
    fsrUnfold.setSysMeanValues()
    fsrUnfold.setSysError()
    fsrUnfold.setTotSysError()

In [21]:
fsrUnfold.doStatUnfold() # NOTE: doStatUnfold() must be done after setMeanValues() and before any other processes
                                   # Don't know why exactly...

In [22]:
fsrUnfold.setStatError() # Update stat. error

# Acceptance correction 

In [23]:
if phase_space == "Fiducial":
    fsrUnfold.doAcceptance(doSys, True)
    fsrUnfold.setAcceptMeanValues()
    if doSys:
        fsrUnfold.setAcceptSysMeanValues()
        fsrUnfold.setAcceptSysError()
        fsrUnfold.setAcceptTotSysError()

In [24]:
#pd_fsr_unfolded_mass, pd_fsr_unfolded_pt = fsrUnfold.makeMeanDataFrames("Unfolded", systematics)
#pd_fsr_acc_mass, pd_fsr_acc_pt = fsrUnfold.makeMeanDataFrames("Acceptance", systematics)
#
#pd_fsr_unfolded_mass.to_csv(year+"_"+channel+"_mass_fsr_unf.csv")
#pd_fsr_unfolded_pt.to_csv(year+"_"+channel+"_pt_fsr_unf.csv")
#
#pd_fsr_acc_mass.to_csv(year+"_"+channel+"_mass_fsr_acc.csv")
#pd_fsr_acc_pt.to_csv(year+"_"+channel+"_pt_fsr_acc.csv")